## Importar librerías
   

In [29]:
import pandas as pd
import numpy as np
import os
from data_util import calcular_ICA

## Cargar datos


In [30]:
file_names = os.listdir('../data/aire/')
file_names

['abr_mo22.csv',
 'ago_mo22.csv',
 'dic_mo22.csv',
 'ene_mo22.csv',
 'feb_mo22.csv',
 'jul_mo22.csv',
 'jun_mo22.csv',
 'mar_mo22.csv',
 'may_mo22.csv',
 'nov_mo22.csv',
 'oct_mo22.csv',
 'sep_mo22.csv']

## Crear único dataset con los los datos del año 2022
Recorrer la lista files_names para crear un único dataset


In [31]:
df_aire = pd.DataFrame()
for file_name in file_names:
    df_current = pd.read_csv('../data/aire/' + file_name, delimiter=';')
    print(file_name, df_current.shape)
    df_aire = pd.concat([df_aire, df_current], ignore_index=True)
df_aire.head(3)

abr_mo22.csv (3951, 56)
ago_mo22.csv (3960, 56)
dic_mo22.csv (4092, 56)
ene_mo22.csv (3929, 56)
feb_mo22.csv (3556, 56)
jul_mo22.csv (3968, 56)
jun_mo22.csv (3872, 56)
mar_mo22.csv (3946, 56)
may_mo22.csv (4059, 56)
nov_mo22.csv (3960, 56)
oct_mo22.csv (4092, 56)
sep_mo22.csv (3838, 56)


,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2022,4,1,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
1,28,79,4,1,28079004_1_38,2022,4,2,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
2,28,79,4,1,28079004_1_38,2022,4,3,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V


## Seleccionar datos:

Nos quedamos con los parámetros que nos interesan: SO2: Dióxido de Azufre, NO2: Dióxido de Nitrógeno, PM2.5: Partículas < 2.5 μm, PM10: Partículas < 10 μm, O3: Ozono

In [32]:
df_aire = df_aire[(df_aire['MAGNITUD'] == 1) | (df_aire['MAGNITUD'] == 8) | (df_aire['MAGNITUD'] == 9) | (df_aire['MAGNITUD'] == 10) | (df_aire['MAGNITUD'] == 14)]
df_aire.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2022,4,1,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
1,28,79,4,1,28079004_1_38,2022,4,2,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
2,28,79,4,1,28079004_1_38,2022,4,3,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
3,28,79,4,1,28079004_1_38,2022,4,4,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
4,28,79,4,1,28079004_1_38,2022,4,5,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V


## Nos quedamos con los parámetros de las estaciones que nos interesan:

8, 18, 24, 35, 36

In [33]:
df_aire  = df_aire[(df_aire['ESTACION'] == 8) | (df_aire['ESTACION'] == 18) | (df_aire['ESTACION'] == 24 ) 
                | (df_aire['ESTACION'] == 35) | (df_aire['ESTACION'] == 36) ]
df_aire.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
150,28,79,8,1,28079008_1_38,2022,4,1,4.0,V,...,4.0,V,4.0,V,4.0,V,4.0,V,4.0,V
151,28,79,8,1,28079008_1_38,2022,4,2,4.0,V,...,5.0,V,4.0,V,4.0,V,4.0,V,4.0,V
152,28,79,8,1,28079008_1_38,2022,4,3,4.0,V,...,4.0,V,4.0,V,4.0,V,4.0,V,4.0,V
153,28,79,8,1,28079008_1_38,2022,4,4,4.0,V,...,4.0,V,4.0,V,4.0,V,4.0,V,4.0,V
154,28,79,8,1,28079008_1_38,2022,4,5,4.0,V,...,4.0,V,4.0,V,4.0,V,4.0,V,4.0,V


## Crear una nueva columna fecha
Con formato datetime

In [34]:
df_aire['FECHA'] = pd.to_datetime(df_aire[['ANO','MES','DIA']].rename(columns = {'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY'}))

## Preparación de los datos de salida:
* Extraer los datos de las medidas tomadas
* Crear dataset auxiliar con los datos extraídos y las nuevas   columnas creadas.

In [35]:
data_columns = [ 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24'] # feature
veri_columns = [ 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']

hora_column = np.arange(1,25) # feature label

data = pd.DataFrame(columns=['HORA', 'DATO', 'VERIFICADO'])
aux = pd.DataFrame(columns=['ANO', 'MES', 'DIA', 'HORA', 'DATO', 'VERIFICADO', 'ESTACION']) # aux de próxima fila a ser añadida
for  (h,d,v) in zip(hora_column, data_columns, veri_columns): 
    aux
    aux['DATO'] = df_aire [ data_columns[h-1] ] 
    aux[['CODIGO', 'MAGNITUD', 'TECNICA']] = df_aire['PUNTO_MUESTREO'].str.split('_', n=2, expand=True)
    aux['VERIFICADO'] = df_aire [ veri_columns[h-1] ] 
    aux['HORA'] = h-1
    aux[['ANO', 'MES', 'DIA']] = df_aire[['ANO','MES','DIA']]
    #aux['MAGNITUD'] = df_aire ['MAGNITUD']
    aux['ESTACION'] = df_aire ['ESTACION']
    data = pd.concat([data, aux], ignore_index=True)
data = data.astype({'HORA': 'int32'}, copy = False)

data.head()

,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,CODIGO,MAGNITUD,TECNICA
0,0,4.0,V,2022.0,4.0,1.0,8.0,28079008,1,38
1,0,4.0,V,2022.0,4.0,2.0,8.0,28079008,1,38
2,0,4.0,V,2022.0,4.0,3.0,8.0,28079008,1,38
3,0,4.0,V,2022.0,4.0,4.0,8.0,28079008,1,38
4,0,4.0,V,2022.0,4.0,5.0,8.0,28079008,1,38


## Añadir la columna fecha en formato datetime al nuevo dataset 

In [36]:
data['FECHA'] = pd.to_datetime(data[['ANO','MES','DIA','HORA']].rename(columns = {'ANO': 'year', 'MES': 'month','DIA': 'day', 'HORA': 'hour'}))
data.head()

,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,CODIGO,MAGNITUD,TECNICA,FECHA
0,0,4.0,V,2022.0,4.0,1.0,8.0,28079008,1,38,2022-04-01
1,0,4.0,V,2022.0,4.0,2.0,8.0,28079008,1,38,2022-04-02
2,0,4.0,V,2022.0,4.0,3.0,8.0,28079008,1,38,2022-04-03
3,0,4.0,V,2022.0,4.0,4.0,8.0,28079008,1,38,2022-04-04
4,0,4.0,V,2022.0,4.0,5.0,8.0,28079008,1,38,2022-04-05


## Ordenar los datos por orden cronológico

In [37]:
data = data.sort_values(['FECHA','HORA'])
data

,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,CODIGO,MAGNITUD,TECNICA,FECHA
1652,0,4.00,V,2022.0,1.0,1.0,8.0,28079008,1,38,2022-01-01 00:00:00
1683,0,35.00,V,2022.0,1.0,1.0,8.0,28079008,8,8,2022-01-01 00:00:00
1714,0,15.00,V,2022.0,1.0,1.0,8.0,28079008,9,47,2022-01-01 00:00:00
1745,0,16.00,V,2022.0,1.0,1.0,8.0,28079008,10,47,2022-01-01 00:00:00
1776,0,18.39,V,2022.0,1.0,1.0,8.0,28079008,14,6,2022-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
151855,23,54.00,V,2022.0,12.0,31.0,35.0,28079035,8,8,2022-12-31 23:00:00
151886,23,4.97,V,2022.0,12.0,31.0,35.0,28079035,14,6,2022-12-31 23:00:00
151917,23,1.00,V,2022.0,12.0,31.0,36.0,28079036,1,38,2022-12-31 23:00:00
151948,23,38.00,V,2022.0,12.0,31.0,36.0,28079036,8,8,2022-12-31 23:00:00


## Marcar datos no válidos en el DataFrame:
* Datos no verificados

In [38]:
data.loc[data['VERIFICADO'] != 'V', ['DATO']] = np.nan
data.loc[data['VERIFICADO'] != 'V']

,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,CODIGO,MAGNITUD,TECNICA,FECHA
1838,0,NaN,N,2022.0,1.0,1.0,18.0,28079018,10,47,2022-01-01 00:00:00
8374,1,NaN,N,2022.0,1.0,1.0,18.0,28079018,10,47,2022-01-01 01:00:00
14910,2,NaN,N,2022.0,1.0,1.0,18.0,28079018,10,47,2022-01-01 02:00:00
21633,3,NaN,N,2022.0,1.0,2.0,35.0,28079035,1,38,2022-01-02 03:00:00
145817,22,NaN,N,2022.0,1.0,2.0,35.0,28079035,1,38,2022-01-02 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...
99255,15,NaN,N,2022.0,12.0,29.0,8.0,28079008,10,47,2022-12-29 15:00:00
105760,16,NaN,N,2022.0,12.0,29.0,8.0,28079008,9,47,2022-12-29 16:00:00
105791,16,NaN,N,2022.0,12.0,29.0,8.0,28079008,10,47,2022-12-29 16:00:00
112296,17,NaN,N,2022.0,12.0,29.0,8.0,28079008,9,47,2022-12-29 17:00:00


## Borrar columna verificado

In [39]:
data = data.drop('VERIFICADO', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156864 entries, 1652 to 151979
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   HORA      156864 non-null  int32         
 1   DATO      150853 non-null  float64       
 2   ANO       156864 non-null  float64       
 3   MES       156864 non-null  float64       
 4   DIA       156864 non-null  float64       
 5   ESTACION  156864 non-null  float64       
 6   CODIGO    156864 non-null  object        
 7   MAGNITUD  156864 non-null  object        
 8   TECNICA   156864 non-null  object        
 9   FECHA     156864 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int32(1), object(3)
memory usage: 12.6+ MB


## Añadir columnas que faltan y formatearlas,

In [40]:
data = data.set_index(['CODIGO','FECHA', 'ANO', 'MES', 'DIA', 'HORA', 'ESTACION','MAGNITUD'])['DATO'].unstack(level=-1).reset_index().rename_axis(None)
data.head()

MAGNITUD,CODIGO,FECHA,ANO,MES,DIA,HORA,ESTACION,1,10,14,8,9
0,28079008,2022-01-01 00:00:00,2022.0,1.0,1.0,0,8.0,4.0,16.0,18.39,35.0,15.0
1,28079008,2022-01-01 01:00:00,2022.0,1.0,1.0,1,8.0,4.0,27.0,8.18,48.0,25.0
2,28079008,2022-01-01 02:00:00,2022.0,1.0,1.0,2,8.0,3.0,27.0,7.05,47.0,25.0
3,28079008,2022-01-01 03:00:00,2022.0,1.0,1.0,3,8.0,3.0,17.0,14.22,38.0,15.0
4,28079008,2022-01-01 04:00:00,2022.0,1.0,1.0,4,8.0,3.0,5.0,24.66,26.0,3.0


## Renombrar columnas con los contaminantes

In [41]:
data = data.rename(columns={'1': 'SO2', '10': 'PM10', '8': 'NO2', '9': 'PM2.5',  '14': 'O3'}, copy = False, errors = 'raise')
data.head()

MAGNITUD,CODIGO,FECHA,ANO,MES,DIA,HORA,ESTACION,SO2,PM10,O3,NO2,PM2.5
0,28079008,2022-01-01 00:00:00,2022.0,1.0,1.0,0,8.0,4.0,16.0,18.39,35.0,15.0
1,28079008,2022-01-01 01:00:00,2022.0,1.0,1.0,1,8.0,4.0,27.0,8.18,48.0,25.0
2,28079008,2022-01-01 02:00:00,2022.0,1.0,1.0,2,8.0,3.0,27.0,7.05,47.0,25.0
3,28079008,2022-01-01 03:00:00,2022.0,1.0,1.0,3,8.0,3.0,17.0,14.22,38.0,15.0
4,28079008,2022-01-01 04:00:00,2022.0,1.0,1.0,4,8.0,3.0,5.0,24.66,26.0,3.0


## Añadir columna para el día de la semana

In [42]:
dates = data['FECHA'].dt.to_pydatetime()
days_of_week = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
data['DIA_SEMANA'] = np.array([days_of_week[d.weekday()] for d in dates])
data.head()

MAGNITUD,CODIGO,FECHA,ANO,MES,DIA,HORA,ESTACION,SO2,PM10,O3,NO2,PM2.5,DIA_SEMANA
0,28079008,2022-01-01 00:00:00,2022.0,1.0,1.0,0,8.0,4.0,16.0,18.39,35.0,15.0,Sabado
1,28079008,2022-01-01 01:00:00,2022.0,1.0,1.0,1,8.0,4.0,27.0,8.18,48.0,25.0,Sabado
2,28079008,2022-01-01 02:00:00,2022.0,1.0,1.0,2,8.0,3.0,27.0,7.05,47.0,25.0,Sabado
3,28079008,2022-01-01 03:00:00,2022.0,1.0,1.0,3,8.0,3.0,17.0,14.22,38.0,15.0,Sabado
4,28079008,2022-01-01 04:00:00,2022.0,1.0,1.0,4,8.0,3.0,5.0,24.66,26.0,3.0,Sabado


In [43]:
dia_semana = data.pop('DIA_SEMANA')
data.insert(5, 'DIA_SEMANA', dia_semana)
data.head()

MAGNITUD,CODIGO,FECHA,ANO,MES,DIA,DIA_SEMANA,HORA,ESTACION,SO2,PM10,O3,NO2,PM2.5
0,28079008,2022-01-01 00:00:00,2022.0,1.0,1.0,Sabado,0,8.0,4.0,16.0,18.39,35.0,15.0
1,28079008,2022-01-01 01:00:00,2022.0,1.0,1.0,Sabado,1,8.0,4.0,27.0,8.18,48.0,25.0
2,28079008,2022-01-01 02:00:00,2022.0,1.0,1.0,Sabado,2,8.0,3.0,27.0,7.05,47.0,25.0
3,28079008,2022-01-01 03:00:00,2022.0,1.0,1.0,Sabado,3,8.0,3.0,17.0,14.22,38.0,15.0
4,28079008,2022-01-01 04:00:00,2022.0,1.0,1.0,Sabado,4,8.0,3.0,5.0,24.66,26.0,3.0


In [44]:
data.reset_index(drop=True)
data.head()

MAGNITUD,CODIGO,FECHA,ANO,MES,DIA,DIA_SEMANA,HORA,ESTACION,SO2,PM10,O3,NO2,PM2.5
0,28079008,2022-01-01 00:00:00,2022.0,1.0,1.0,Sabado,0,8.0,4.0,16.0,18.39,35.0,15.0
1,28079008,2022-01-01 01:00:00,2022.0,1.0,1.0,Sabado,1,8.0,4.0,27.0,8.18,48.0,25.0
2,28079008,2022-01-01 02:00:00,2022.0,1.0,1.0,Sabado,2,8.0,3.0,27.0,7.05,47.0,25.0
3,28079008,2022-01-01 03:00:00,2022.0,1.0,1.0,Sabado,3,8.0,3.0,17.0,14.22,38.0,15.0
4,28079008,2022-01-01 04:00:00,2022.0,1.0,1.0,Sabado,4,8.0,3.0,5.0,24.66,26.0,3.0


## CARGAR DATOS DE LAS ESTACIONES DE MEDIDA


In [45]:
df_estaciones = pd.read_csv('../data/estaciones.csv', delimiter=';')
df_estaciones.head()

,CODIGO,CODIGO_CORTO,ESTACION,DIRECCION,LONGITUD_ETRS89,LATITUD_ETRS89,ALTITUD,COD_TIPO,NOM_TIPO,NO2,...,BTX,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,Fecha alta,COORDENADA_X_ETRS89,COORDENADA_Y_ETRS89,LONGITUD,LATITUD
0,28079004,4,Plaza de España,Plaza de España,"3°42'43.91""O","40°25'25.98""N",637,UT,Urbana tráfico,X,...,NaN,273600,PLAZA,DE,ESPAÑA,01/12/1998,"439579,3291","4475049,263",-3.712257,40.423882
1,28079008,8,Escuelas Aguirre,Entre C/ Alcalá y C/ O’ Donell,"3°40'56.22""O","40°25'17.63""N",672,UT,Urbana tráfico,X,...,X,18900,CALLE,DE,ALCALA,01/12/1998,"442117,2366","4474770,696",-3.682316,40.421553
2,28079011,11,Ramón y Cajal,Avda. Ramón y Cajal esq. C/ Príncipe de Vergara,"3°40'38.50""O","40°27'5.29""N",708,UT,Urbana tráfico,X,...,X,610450,CALLE,DEL,PRINCIPE DE VERGARA,01/12/1998,"442564,0457","4478088,595",-3.677349,40.451473
3,28079016,16,Arturo Soria,C/ Arturo Soria esq. C/ Vizconde de los Asilos,"3°38'21.17""O","40°26'24.20""N",695,UF,Urbana fondo,X,...,NaN,798700,CALLE,DEL,VIZCONDE DE LOS ASILOS,01/12/1998,"445786,1729","4476796,019",-3.639242,40.440046
4,28079017,17,Villaverde,C/ Juan Peñalver,"3°42'47.89""O","40°20'49.74""N",601,UF,Urbana fondo,X,...,NaN,417200,CALLE,DE,JUAN PEÑALVER,01/12/1998,"439420,7015","4466532,455",-3.713317,40.347147


## Crear en data las siguentes columnas y rellenarlas con los datos de las estaciones
NOM_TIPO
LATITUD
LONGITUD

In [46]:
# Convertir la columna 'CODIGO' en ambos DataFrame a tipo int64
df_estaciones['CODIGO'] = df_estaciones['CODIGO'].astype(int)
data['CODIGO'] = data['CODIGO'].astype(int)

# Realizar un merge basado en la columna 'CODIGO' para obtener la columna 'NOM_TIPO', 'LATITUD', 'LONGITUD'
data = data.merge(df_estaciones[['CODIGO', 'NOM_TIPO', 'LATITUD', 'LONGITUD']], on='CODIGO', how='left')
data.head(2)

,CODIGO,FECHA,ANO,MES,DIA,DIA_SEMANA,HORA,ESTACION,SO2,PM10,O3,NO2,PM2.5,NOM_TIPO,LATITUD,LONGITUD
0,28079008,2022-01-01 00:00:00,2022.0,1.0,1.0,Sabado,0,8.0,4.0,16.0,18.39,35.0,15.0,Urbana tráfico,40.421553,-3.682316
1,28079008,2022-01-01 01:00:00,2022.0,1.0,1.0,Sabado,1,8.0,4.0,27.0,8.18,48.0,25.0,Urbana tráfico,40.421553,-3.682316


In [47]:
data.columns

Index(['CODIGO', 'FECHA', 'ANO', 'MES', 'DIA', 'DIA_SEMANA', 'HORA',
       'ESTACION', 'SO2', 'PM10', 'O3', 'NO2', 'PM2.5', 'NOM_TIPO', 'LATITUD',
       'LONGITUD'],
      dtype='object')

## Datos no válidos

In [48]:
data.isnull().sum()

CODIGO            0
FECHA             0
ANO               0
MES               0
DIA               0
DIA_SEMANA        0
HORA              0
ESTACION          0
SO2           18243
PM10          11520
O3             9216
NO2             439
PM2.5         27409
NOM_TIPO          0
LATITUD           0
LONGITUD          0
dtype: int64

## Calcular ICA 

A partir de la función calcular_ICA(data)

In [49]:
# Llamar a la función y asignar el resultado a una nueva variable
data = calcular_ICA(data)

# Imprimir el dataframe con las nuevas columnas
data.head(2)

,CODIGO,FECHA,ANO,MES,DIA,DIA_SEMANA,HORA,ESTACION,SO2,PM10,...,NO2,PM2.5,NOM_TIPO,LATITUD,LONGITUD,ICA_SO2,ICA_PM10,ICA_O3,ICA_NO2,ICA
0,28079008,2022-01-01 00:00:00,2022.0,1.0,1.0,Sabado,0,8.0,4.0,16.0,...,35.0,15.0,Urbana tráfico,40.421553,-3.682316,3.2,32.0,15.32,87.5,34.51
1,28079008,2022-01-01 01:00:00,2022.0,1.0,1.0,Sabado,1,8.0,4.0,27.0,...,48.0,25.0,Urbana tráfico,40.421553,-3.682316,3.2,54.0,6.82,120.0,46.00


In [50]:
data.isnull().sum()

CODIGO            0
FECHA             0
ANO               0
MES               0
DIA               0
DIA_SEMANA        0
HORA              0
ESTACION          0
SO2           18243
PM10          11520
O3             9216
NO2             439
PM2.5         27409
NOM_TIPO          0
LATITUD           0
LONGITUD          0
ICA_SO2       18243
ICA_PM10      11520
ICA_O3         9216
ICA_NO2         439
ICA             218
dtype: int64

In [51]:
data.head()

,CODIGO,FECHA,ANO,MES,DIA,DIA_SEMANA,HORA,ESTACION,SO2,PM10,...,NO2,PM2.5,NOM_TIPO,LATITUD,LONGITUD,ICA_SO2,ICA_PM10,ICA_O3,ICA_NO2,ICA
0,28079008,2022-01-01 00:00:00,2022.0,1.0,1.0,Sabado,0,8.0,4.0,16.0,...,35.0,15.0,Urbana tráfico,40.421553,-3.682316,3.2,32.0,15.32,87.5,34.51
1,28079008,2022-01-01 01:00:00,2022.0,1.0,1.0,Sabado,1,8.0,4.0,27.0,...,48.0,25.0,Urbana tráfico,40.421553,-3.682316,3.2,54.0,6.82,120.0,46.00
2,28079008,2022-01-01 02:00:00,2022.0,1.0,1.0,Sabado,2,8.0,3.0,27.0,...,47.0,25.0,Urbana tráfico,40.421553,-3.682316,2.4,54.0,5.88,117.5,44.94
3,28079008,2022-01-01 03:00:00,2022.0,1.0,1.0,Sabado,3,8.0,3.0,17.0,...,38.0,15.0,Urbana tráfico,40.421553,-3.682316,2.4,34.0,11.85,95.0,35.81
4,28079008,2022-01-01 04:00:00,2022.0,1.0,1.0,Sabado,4,8.0,3.0,5.0,...,26.0,3.0,Urbana tráfico,40.421553,-3.682316,2.4,10.0,20.55,65.0,24.49


In [52]:
data.to_csv('../data/auxiliar/aire_2022.csv') # Salvado con todos los datos